In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle
! mkdir ~/../content/data

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download gpiosenka/100-bird-species

 99% 1.90G/1.91G [00:20<00:00, 106MB/s]
100% 1.91G/1.91G [00:20<00:00, 100MB/s]


In [ ]:
! unzip 100-bird-species.zip -d ~/../content/data

Streaming output truncated to the last 5000 lines.
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/028.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/029.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/030.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/031.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/032.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/033.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/034.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/035.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/036.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/037.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/038.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/039.jpg  
  inflating: /root/../content/data/train/WHITE NECKED RAVEN/040.jpg  
  inflating: /root/../content/data/trai

In [ ]:
# imports
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
birdcsv = pd.read_csv('/content/data/birds.csv')
print(birdcsv.columns)

bird_count = {}
birds = birdcsv['labels'].copy()
for bird in birds:
  bird_count[bird] = bird_count.get(bird, 0) + 1

total_samples = birdcsv.shape[0]
print("Total samples: ", total_samples)
percent = []
bird_labels = bird_count.keys()

for bird, count in bird_count.items():
  print("Bird:", bird + ",", "Samples:", count)

list_of_counts = list(map(lambda x: (x, bird_count[x]), bird_count.keys()))
print(len(list_of_counts))
print(sorted(list_of_counts, key=lambda x: x[1], reverse=True)[0:5])



Index(['class id', 'filepaths', 'labels', 'data set', 'scientific name'], dtype='object')
Total samples:  87874
Bird: ABBOTTS BABBLER, Samples: 173
Bird: ABBOTTS BOOBY, Samples: 190
Bird: ABYSSINIAN GROUND HORNBILL, Samples: 189
Bird: AFRICAN CROWNED CRANE, Samples: 145
Bird: AFRICAN EMERALD CUCKOO, Samples: 164
Bird: AFRICAN FIREFINCH, Samples: 147
Bird: AFRICAN OYSTER CATCHER, Samples: 165
Bird: AFRICAN PIED HORNBILL, Samples: 197
Bird: AFRICAN PYGMY GOOSE, Samples: 189
Bird: ALBATROSS, Samples: 142
Bird: ALBERTS TOWHEE, Samples: 171
Bird: ALEXANDRINE PARAKEET, Samples: 175
Bird: ALPINE CHOUGH, Samples: 176
Bird: ALTAMIRA YELLOWTHROAT, Samples: 160
Bird: AMERICAN AVOCET, Samples: 189
Bird: AMERICAN BITTERN, Samples: 180
Bird: AMERICAN COOT, Samples: 168
Bird: AMERICAN FLAMINGO, Samples: 174
Bird: AMERICAN GOLDFINCH, Samples: 143
Bird: AMERICAN KESTREL, Samples: 165
Bird: AMERICAN PIPIT, Samples: 189
Bird: AMERICAN REDSTART, Samples: 149
Bird: AMERICAN ROBIN, Samples: 157
Bird: AMERIC

In [ ]:
#hyperparameters

input_size = 224
hidden_size = 1024 #arbitrary
num_classes = 515
num_epochs = 14 #arbitrary
batch_size = 200 #arbitrary
learning_rate = 0.0003

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_dir = "/content/data/"

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ]),
}

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(("../" + data_dir + "/" + x + "/"), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

In [ ]:
# convolutional network class

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=11, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=7, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            )
        self.layer4 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(32,32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(32,32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(64,32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(15*15*32, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out1 = self.layer2(out)
        out = self.layer3(out1)
        out += out1
        out = self.layer4(out)
        out1 = self.layer5(out)
        out = self.layer6(out1)
        out += out1
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model_conv = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate)

# Train the model_conv
train_loader = dataloaders_dict['train']
test_loader = dataloaders_dict['valid']
total_step = len(train_loader)
test_acc_list, train_acc_list = [], []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model_conv(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 12 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Test the model_conv
    model_conv.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_conv(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model_conv on the 25 test images: {} %'.format(100 * correct / total))
        test_acc_list.append(100 * correct / total)

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_conv(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model_conv on the train images: {} %'.format(100 * correct / total))
        train_acc_list.append(100 * correct / total)



plt.plot(train_acc_list, '-b', label='train acc')
plt.plot(test_acc_list, '-r', label='test acc')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.xticks(rotation=60)
plt.title('Accuracy ~ Epoch')
# plt.savefig('assets/accr_{}.png'.format(cfg_idx))
plt.show()


Epoch [1/14], Step [12/414], Loss: 6.3195
Epoch [1/14], Step [24/414], Loss: 6.2140
Epoch [1/14], Step [36/414], Loss: 6.0412
Epoch [1/14], Step [48/414], Loss: 5.8993
Epoch [1/14], Step [60/414], Loss: 5.7883
Epoch [1/14], Step [72/414], Loss: 5.7963
Epoch [1/14], Step [84/414], Loss: 5.8024
Epoch [1/14], Step [96/414], Loss: 5.6786
Epoch [1/14], Step [108/414], Loss: 5.5676
Epoch [1/14], Step [120/414], Loss: 5.7413
Epoch [1/14], Step [132/414], Loss: 5.6243
Epoch [1/14], Step [144/414], Loss: 5.5306
Epoch [1/14], Step [156/414], Loss: 5.2837
Epoch [1/14], Step [168/414], Loss: 5.2918
Epoch [1/14], Step [180/414], Loss: 5.2439
Epoch [1/14], Step [192/414], Loss: 5.1586
Epoch [1/14], Step [204/414], Loss: 5.1455
Epoch [1/14], Step [216/414], Loss: 5.1602
Epoch [1/14], Step [228/414], Loss: 4.9675
Epoch [1/14], Step [240/414], Loss: 5.2940
Epoch [1/14], Step [252/414], Loss: 4.9681
Epoch [1/14], Step [264/414], Loss: 4.9892
Epoch [1/14], Step [276/414], Loss: 4.7535
Epoch [1/14], Step 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Epoch [1/14], Step [408/414], Loss: 4.7034
Test Accuracy of the model_conv on the 25 test images: 19.339805825242717 %


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Test Accuracy of the model_conv on the train images: 12.570717083313186 %
Epoch [2/14], Step [12/414], Loss: 5.9865
Epoch [2/14], Step [24/414], Loss: 5.6058
Epoch [2/14], Step [36/414], Loss: 5.3915
Epoch [2/14], Step [48/414], Loss: 5.2629
Epoch [2/14], Step [60/414], Loss: 5.1396
Epoch [2/14], Step [72/414], Loss: 5.1300
Epoch [2/14], Step [84/414], Loss: 5.1158
Epoch [2/14], Step [96/414], Loss: 4.9250
Epoch [2/14], Step [108/414], Loss: 4.8746
Epoch [2/14], Step [120/414], Loss: 5.0588
Epoch [2/14], Step [132/414], Loss: 4.8358
Epoch [2/14], Step [144/414], Loss: 4.9790
Epoch [2/14], Step [156/414], Loss: 4.9556
Epoch [2/14], Step [168/414], Loss: 4.7893
Epoch [2/14], Step [180/414], Loss: 4.9170
Epoch [2/14], Step [192/414], Loss: 4.7951
Epoch [2/14], Step [204/414], Loss: 4.5875
Epoch [2/14], Step [216/414], Loss: 4.5365
Epoch [2/14], Step [228/414], Loss: 4.6957
Epoch [2/14], Step [240/414], Loss: 4.5837
Epoch [2/14], Step [252/414], Loss: 4.5597
Epoch [2/14], Step [264/414], L

...

Epoch [8/14], Step [300/414], Loss: 3.1502


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Epoch [8/14], Step [312/414], Loss: 3.2932
Epoch [8/14], Step [324/414], Loss: 3.1990
Epoch [8/14], Step [336/414], Loss: 3.5771
Epoch [8/14], Step [348/414], Loss: 3.1655
Epoch [8/14], Step [360/414], Loss: 3.1029
Epoch [8/14], Step [372/414], Loss: 3.4983
Epoch [8/14], Step [384/414], Loss: 3.1817
Epoch [8/14], Step [396/414], Loss: 3.1567
Epoch [8/14], Step [408/414], Loss: 3.1285
Test Accuracy of the model_conv on the 25 test images: 52.85436893203884 %
Test Accuracy of the model_conv on the train images: 35.34766210531406 %
Epoch [9/14], Step [12/414], Loss: 3.0088
Epoch [9/14], Step [24/414], Loss: 3.3125
Epoch [9/14], Step [36/414], Loss: 2.7736
Epoch [9/14], Step [48/414], Loss: 3.1145
Epoch [9/14], Step [60/414], Loss: 3.4259
Epoch [9/14], Step [72/414], Loss: 3.0432
Epoch [9/14], Step [84/414], Loss: 3.1885
Epoch [9/14], Step [96/414], Loss: 3.1323
Epoch [9/14], Step [108/414], Loss: 2.9465
Epoch [9/14], Step [120/414], Loss: 3.1174
Epoch [9/14], Step [132/414], Loss: 3.1275


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionErrorException ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f39881320d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
:     self._shutdown_workers()can only test a child process

  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Test Accuracy of the model_conv on the train images: 35.36700352980997 %
Epoch [10/14], Step [12/414], Loss: 3.0042
Epoch [10/14], Step [24/414], Loss: 3.4953
Epoch [10/14], Step [36/414], Loss: 3.2445
Epoch [10/14], Step [48/414], Loss: 3.3912
Epoch [10/14], Step [60/414], Loss: 3.0117
